In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

In [3]:
# Load data
data = pd.read_csv('/content/drive/My Drive/M2 GL/PFE/Data/hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing_v3.csv')
safa_data = pd.read_csv('/content/drive/My Drive/M2 GL/PFE/Data/safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing.csv')

In [4]:
export_dir = '/content/drive/My Drive/M2 GL/PFE/new Tajweed rule model/AI models/v5'

In [5]:
abdul_basit = data[data['recitor_en'] == 'Abdul Basit']
yassin_aljazaery = data[data['recitor_en'] == 'Yassin Al Jazaery']
ibrahim_aldosary = data[data['recitor_en'] == 'Ibrahim_Aldosary']
al_husary = data[data['recitor_en'] == 'Al husary']

In [6]:
splitted_data_info_np = np.empty((0, 6))
models_information_np = np.empty((0, 5))

In [7]:
max_Y = {'madd_6_Lazim': 2, 'madd_246': 3, 'madd_6': 6, 'madd_2': 5, 'Ikhfaa': 9, 'Idgham': 13, 'tafkhim': 24, 'qalqala': 6, 'imala': 7}
max_X = 8000

In [8]:
def data_preparation(reciter_data, tajweed_rule):
  data_filtered = reciter_data[reciter_data[tajweed_rule].apply(lambda x: x != '[-1]')]

  # Extract 'mfcc' and tajweed_rule columns as lists of strings
  X_raw = data_filtered['mfcc'].astype(str).tolist()
  Y_raw = data_filtered[tajweed_rule].astype(str).tolist()

  # Preprocess the input data
  X = [tf.constant(eval(x)) for x in X_raw]
  Y = [tf.constant(eval(x)) for x in Y_raw]

  # Pad sequences in Y and in X to ensure all have the same length
  Y_padded = tf.keras.preprocessing.sequence.pad_sequences(Y, maxlen=max_Y[tajweed_rule], padding='post', dtype='int32', value=-1)
  X_padded = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_X, padding='post', dtype='float32')

  # Split the data into training and testing sets
  X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_padded, test_size=0.2, random_state=10)
  return X_train, X_test, Y_train, Y_test

In [9]:
def tajweed_rule_model(reciter1, reciter2, reciter3, reciter4, not_exp, tajweed_rule):
  global splitted_data_info_np, models_information_np, data

  # data preparation
  reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test = data_preparation(reciter1, tajweed_rule)
  reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test = data_preparation(reciter2, tajweed_rule)
  reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test = data_preparation(reciter3, tajweed_rule)
  reciter4_X_train, reciter4_X_test, reciter4_Y_train, reciter4_Y_test = data_preparation(reciter4, tajweed_rule)
  not_exp_X_train, not_exp_X_test, not_exp_Y_train, not_exp_Y_test = data_preparation(not_exp, tajweed_rule)

  # Update splitted_data_info with information about each reciter
  for reciter_X_train, reciter_X_test, reciter_Y_train, reciter_Y_test, reciter_data in [
      (reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test, reciter1),
      (reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test, reciter2),
      (reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test, reciter3),
      (reciter4_X_train, reciter4_X_test, reciter4_Y_train, reciter4_Y_test, reciter4),
      (not_exp_X_train, not_exp_X_test, not_exp_Y_train, not_exp_Y_test, not_exp)]:

      splitted_data_info_np = np.append(splitted_data_info_np, [[
              tajweed_rule,
              reciter_data.iloc[0]['recitor_en'],
              len(reciter_X_train),
              len(reciter_X_test),
              len(reciter_Y_train),
              len(reciter_Y_test)
              ]], axis=0)

  # concatenate data
  # training data
  X_train = np.concatenate([reciter1_X_train, reciter2_X_train, reciter3_X_train, reciter4_X_train, not_exp_X_train], axis=0)
  Y_train = np.concatenate([reciter1_Y_train, reciter2_Y_train, reciter3_Y_train, reciter4_Y_train, not_exp_Y_train], axis=0)

  # testing data
  X_test = np.concatenate([reciter1_X_test, reciter2_X_test, reciter3_X_test, reciter4_X_test, not_exp_X_test], axis=0)
  Y_test = np.concatenate([reciter1_Y_test, reciter2_Y_test, reciter3_Y_test, reciter4_Y_test, not_exp_Y_test], axis=0)

  splitted_data_info_np = np.append(splitted_data_info_np, [[
          tajweed_rule,
          'all reciters',
          len(X_train),
          len(X_test),
          len(Y_train),
          len(Y_test)
          ]], axis=0)

  # Normalize input data by scaling each sequence individually
  scaler = StandardScaler()
  X_train_scaled = np.array([scaler.fit_transform(seq) for seq in X_train])
  X_test_scaled = np.array([scaler.transform(seq) for seq in X_test])

  # Define a simple neural network model
  input_shape = X_train_scaled[0].shape  # Shape of each mfcc sequence
  output_shape = Y_train.shape[1]  # Dimension of output (number of units in output layer)

  input_layer = Input(shape=input_shape)
  flatten_layer = Flatten()(input_layer)  # Flatten the sequence to a 1D vector
  hidden_layer = Dense(64, activation='relu')(flatten_layer)
  output_layer = Dense(output_shape, activation='linear')(hidden_layer)  # Define the output layer with the correct units

  model = Model(inputs=input_layer, outputs=output_layer)

  # Compile the model
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, Y_train, epochs=50, batch_size=32, validation_split=0.1)

  #export the model
  model_filename = f'{tajweed_rule}_tajweed_rule_model'
  model_path = os.path.join(export_dir, model_filename)
  keras.models.save_model(model, model_path)

  # Make predictions on test data
  predictions = model.predict(X_test_scaled)

  # Evaluate the model with adjusted predictions
  predictions[predictions < 0] = -1
  predictions = np.round(predictions).astype('int32')
  loss, accuracy = model.evaluate(X_test_scaled, predictions)

  print(f"Test Loss: {loss:.4f}, Test accuracy : {accuracy:.4f}")
  models_information_np = np.append(models_information_np, [[
          model_filename,
          "{:.4f}".format(loss),
          "{:.4f}".format(accuracy),
          "{:.2f}".format(accuracy*100),
          model_path]], axis=0)

In [10]:
for rule in max_Y.keys():
  tajweed_rule_model(abdul_basit, yassin_aljazaery, ibrahim_aldosary, al_husary, safa_data, rule)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 189.3658 - accuracy: 0.5000 - val_loss: 1468.3739 - val_accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 161ms/step - loss: 4483.3213 - accuracy: 1.0000 - val_loss: 69.5942 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 130ms/step - loss: 644.0039 - accuracy: 0.5000 - val_loss: 260.9345 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 135ms/step - loss: 355.1581 - accuracy: 0.0000e+00 - val_loss: 240.2511 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 131ms/step - loss: 308.6852 - accuracy: 0.0000e+00 - val_loss: 228.7530 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 123ms/step - loss: 209.5018 - accuracy: 0.0000e+00 - val_loss: 215.0124 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 167ms/step - loss: 147.5570 - accuracy: 0.2500 - val_

5/5 [==============================] - 0s 16ms/step - loss: 1.7859 - accuracy: 0.9333
Test Loss: 1.7859, Test accuracy : 0.9333
Epoch 1/50
17/17 [==============================] - 3s 128ms/step - loss: 231.4670 - accuracy: 0.3723 - val_loss: 248.7605 - val_accuracy: 0.4386
Epoch 2/50
17/17 [==============================] - 3s 155ms/step - loss: 195.6420 - accuracy: 0.3489 - val_loss: 273.4591 - val_accuracy: 0.5088
Epoch 3/50
17/17 [==============================] - 2s 138ms/step - loss: 145.0264 - accuracy: 0.4269 - val_loss: 256.2285 - val_accuracy: 0.2807
Epoch 4/50
17/17 [==============================] - 2s 106ms/step - loss: 107.0957 - accuracy: 0.5848 - val_loss: 252.5235 - val_accuracy: 0.4035
Epoch 5/50
17/17 [==============================] - 2s 103ms/step - loss: 90.7646 - accuracy: 0.6179 - val_loss: 255.7717 - val_accuracy: 0.1579
Epoch 6/50
17/17 [==============================] - 2s 104ms/step - loss: 78.0461 - accuracy: 0.6140 - val_loss: 248.9142 - val_accuracy: 0.508

In [11]:
def print_model_summary(loaded_model, tajweed_rule):
  print(f'******* Tajweed rule {tajweed_rule} model *******')
  loaded_model.summary()
  print('\n')

In [12]:
for rule in max_Y.keys():
    model_filename = f'{rule}_tajweed_rule_model'
    model_path = os.path.join(export_dir, model_filename)

    # Load the saved model
    loaded_model = tf.keras.models.load_model(model_path)

    print_model_summary(loaded_model, rule)

******* Tajweed rule madd_6_Lazim model *******
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 13)]        0         
                                                                 
 flatten (Flatten)           (None, 104000)            0         
                                                                 
 dense (Dense)               (None, 64)                6656064   
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 6656194 (25.39 MB)
Trainable params: 6656194 (25.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


******* Tajweed rule madd_246 model *******
Model: "model_1"
__________________________________________

In [13]:
# how data is splitted
columns1 = ['tajweed_rule', 'data_of', 'X_train_nb_samples', 'X_test_nb_samples', 'Y_train_nb_samples', 'X_test_nb_samples']
splitted_data_info = pd.DataFrame(data=splitted_data_info_np, columns=columns1)

# save models information
columns2 = ['Model', 'Loss', 'Accuracy', 'Accuracy %', 'Path_to_the_model']
models_information = pd.DataFrame(data=models_information_np, columns=columns2)

In [14]:
splitted_data_info

,tajweed_rule,data_of,X_train_nb_samples,X_test_nb_samples,Y_train_nb_samples,X_test_nb_samples
0,madd_6_Lazim,Abdul Basit,1,1,1,1
1,madd_6_Lazim,Yassin Al Jazaery,1,1,1,1
2,madd_6_Lazim,Ibrahim_Aldosary,1,1,1,1
3,madd_6_Lazim,Al husary,1,1,1,1
4,madd_6_Lazim,safa,1,1,1,1
5,madd_6_Lazim,all reciters,5,5,5,5
6,madd_246,Abdul Basit,62,16,62,16
7,madd_246,Yassin Al Jazaery,62,16,62,16
8,madd_246,Ibrahim_Aldosary,62,16,62,16
9,madd_246,Al husary,62,16,62,16


In [15]:
models_information

,Model,Loss,Accuracy,Accuracy %,Path_to_the_model
0,madd_6_Lazim_tajweed_rule_model,89.8238,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
1,madd_246_tajweed_rule_model,28.1466,0.9875,98.75,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
2,madd_6_tajweed_rule_model,201.7663,0.9333,93.33,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
3,madd_2_tajweed_rule_model,4.9547,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
4,Ikhfaa_tajweed_rule_model,1.7859,0.9333,93.33,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
5,Idgham_tajweed_rule_model,0.9408,0.9172,91.72,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
6,tafkhim_tajweed_rule_model,70.6740,0.9244,92.44,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
7,qalqala_tajweed_rule_model,6029.3921,0.9800,98.00,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
8,imala_tajweed_rule_model,5.9837,0.9286,92.86,/content/drive/My Drive/M2 GL/PFE/new Tajweed ...
